In [150]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma, poisson

# Récupération des données
Ndoses = 6
Nplates = 3
y = np.array([[15, 16, 16],
              [27, 33, 20],
              [21, 18, 26],
              [41, 38, 27],
              [29, 21, 33],
              [60, 41, 42]])
x = np.array([0, 10, 33, 100, 333, 1000])


In [215]:

# Initialisation
alpha = 0
beta = 0
gamma = 0
tau = 0.1
mu = np.random.normal(0, tau, (6, 3))



In [159]:
print(mu)

[[ 0.03246677 -0.00402394 -0.13076105]
 [-0.01575338  0.06468695 -0.05220653]
 [ 0.07104585 -0.08235794  0.01458342]
 [ 0.07445067 -0.05124446 -0.03421925]
 [-0.0799408  -0.10059937  0.17797019]
 [-0.01817507  0.05681934 -0.01603141]]


In [224]:
import numpy as np

def somme_des_logs(y):
    sommes = []
    for ligne in y:
        somme_ligne = []
        for val in ligne:
            somme_ligne.append(np.log(val))
        sommes.append(somme_ligne)
    return sommes



In [225]:
import math

def som_des_logs(n):
    somme = 0
    for i in range(1, n+1):
        somme += math.log(i)
    return somme


In [226]:
import numpy as np
import scipy.special

def salm(nchain, x, y, alpha, beta, gamma, tau, mu, prop_sd):
  
    # Initialisation
    # res = np.ones((nchain + 1, 4))
    res = []
    res.append([alpha, beta, gamma, 1/np.sqrt(tau)])
    print(res[0][1])
    n = len(x)
    m = y.shape[1]
    
    acc_rates = np.zeros(3) 
    
    for i in range(nchain):

        # Mise à jour de alpha
        alpha = res[i][0]
        prop = np.random.normal(alpha, prop_sd[0])  # marche aléatoire simple
        prop_logmu = mu
        for l in range(n):
            for k in range(m):
                prop_logmu[l, k] = prop + beta * np.log(10 + x[l]) + gamma * x[l] + mu[l, k]
        prop_mu = np.exp(prop_logmu)
        sum_log = somme_des_logs(y)
        top = - ((prop**2) / (2 * 1e-6)) + np.sum(prop * np.log(prop_mu+1e-10) - sum_log - prop_mu)
        bottom = - ((alpha**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
        acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)
        
        if np.random.uniform(0, 1) < min(1, acc_prob):
            alpha = prop
            logmu = prop_logmu
            mu = prop_mu
            acc_rates[0] += 1
        
        # Mise à jour de beta
        print(i)
        beta = res[i][1]
        prop = np.random.normal(beta, prop_sd[1])  # marche aléatoire simple
        prop_logmu = mu
        for l in range(n):
            for k in range(m):
                prop_logmu[l, k] = alpha + prop * np.log(10 + x[l]) + gamma * x[l] + mu[l, k]
        prop_mu = np.exp(prop_logmu)
        sum_log = somme_des_logs(y)
        top = - ((prop**2) / (2 * 1e-6)) + np.sum(prop * np.log(np.abs(prop_mu)+1e-10) - sum_log - prop_mu)
        bottom = - ((beta**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
        acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)
        
        if np.random.uniform(0, 1) < min(1, acc_prob):
            beta = prop
            logmu = prop_logmu
            mu = prop_mu
            acc_rates[1] += 1
        
        # Mise à jour de gamma
        gamma = res[i][2]
        prop = np.random.normal(gamma, prop_sd[2])  # marche aléatoire simple
        prop_logmu = logmu.copy()
        for l in range(n):
            for k in range(m):
                prop_logmu[l, k] = alpha + beta * np.log(10 + x[l]) + prop * x[l] + mu[l, k]
        prop_mu = np.abs(np.exp(prop_logmu))
        sum_log = somme_des_logs(y)
        top = - ((prop**2) / (2 * 1e-6)) + np.sum(prop * np.log(np.abs(prop_mu)+1e-10) - sum_log - prop_mu)
        bottom = - ((gamma**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
        acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)
        
        if np.random.uniform(0, 1) < min(1, acc_prob):
            gamma = prop
            logmu = prop_logmu
            mu = prop_mu
            acc_rates[2] += 1
        
        # Mise à jour de tau
        #tau = np.random.gamma(10e-3 + (n + m) / 2, 1e-3 + 0.5 * np.sum(mu**2))
        tau = 0.1
        print(mu**2)
        print(tau)

        # Mise à jour de mu
        for k in range(n):
            for l in range(m):
                prop = np.random.normal(mu[k, l], prop_sd[3])
                prop_logmu[k, l] = alpha + beta * np.log(10 + x[k]) + gamma * x[k] + prop
                
                top = - (prop**2 / (2*tau)) + np.log(prop**y[k, l]) - som_des_logs(y[k][l])
                bottom = - (mu[k, l]**2/ (2*tau)) + np.log(mu[k, l]**y[k, l]) - som_des_logs(y[k][l])
                acc_prob = np.exp(top - bottom)
              
                if np.random.uniform(0, 1) < min(1, acc_prob):
                    mu[k, l] = prop
        
        # Mise à jour de la chaîne de Markov et de b
        res.append([alpha, beta, gamma, 1/np.sqrt(tau)])
    
    my_list = {"chain": res, "acc_rates": acc_rates}
    return my_list



In [216]:
mu

array([[ 0.01673646, -0.03576919,  0.03298067],
       [-0.02006928,  0.17950656, -0.0426333 ],
       [-0.12503186, -0.09632762,  0.19459843],
       [ 0.12919128, -0.13481751, -0.01975478],
       [ 0.09816098, -0.08875833,  0.0211398 ],
       [-0.05671434, -0.04179002, -0.12101632]])

In [228]:
resultat = salm(int(1e4), x, y, alpha, beta, gamma, tau, mu, prop_sd=[0.1, 0.1, 0.1, 0.001])  # prop_sd choisi pour avoir une apparence de chaîne cohérente

resultat_chain = resultat["chain"][1000:]  # enlever les 1000 premiers (burn-in)

moychain = np.mean(resultat_chain, axis=0)  # moyenne
sdchain = np.std(resultat_chain, axis=0)  # écart-type

print("alpha est estimé à", moychain[0], "avec un écart-type de", sdchain[0])
print("beta est estimé à", moychain[1], "avec un écart-type de", sdchain[1])
print("gamma est estimé à", moychain[2], "avec un écart-type de", sdchain[2])
print("sigma est estimé à", moychain[3], "avec un écart-type de", sdchain[3])



0
0
[[3.10199017 2.89205511 3.17237362]
 [3.00308616 4.02013569 2.9159004 ]
 [2.68304668 2.7775639  4.19846393]
 [3.87393632 2.71434949 3.13340645]
 [3.80186753 2.9508974  3.40525548]
 [3.1547282  3.21569358 2.91428328]]
0.1
1
[[26.2165125  22.97545353 27.17231353]
 [24.73715944 43.64816401 23.36691625]
 [20.18924992 21.55647511 48.18777393]
 [40.91392218 20.62331775 26.87714437]
 [39.69114767 24.08570569 31.70142018]
 [27.40274827 28.55203966 23.68643883]]
0.1
2
[[20.40359441 17.56386349 21.26069355]
 [17.80505627 34.26488141 16.63645091]
 [12.75615447 13.85408507 36.23704816]
 [27.78190425 11.66246903 16.44889473]
 [24.25728776 12.4854658  18.11241906]
 [13.12461463 13.93908746 10.62006261]]
0.1
3
[[18.71240571 15.998317   19.54310526]
 [16.40490913 32.30338471 15.29420663]
 [11.74503502 12.80803418 34.52059814]
 [26.59570267 10.88854373 15.55290804]
 [23.52461662 11.94611406 17.47405263]
 [12.83741029 13.63912615 10.3573832 ]]
0.1
4
[[16.55834481 14.02633432 17.3636731 ]
 [14.437728

C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:28: RuntimeWarning: invalid value encountered in log
  bottom = - ((alpha**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:29: RuntimeWarning: overflow encountered in exp
  acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)


[[ 3360.36706424  1784.62240138  4077.36194026]
 [ 2185.72785928 51761.0313881   1688.91642045]
 [  828.12295099  1069.20888158 79494.69715364]
 [20970.61471375   829.82651514  2326.07633766]
 [13011.40055992  1296.15787636  4022.97239227]
 [ 1857.07417272  2168.95301042  1151.41233761]]
0.1
68
[[ 3372.10135185  1793.0846028   4090.29780272]
 [ 2206.76757019 51863.15103464  1707.32040224]
 [  849.05609084  1092.9264988  79698.6343396 ]
 [21124.09383557   860.49845858  2377.37781946]
 [13178.5647026   1349.34410493  4116.3868802 ]
 [ 1937.63036066  2256.0208794   1215.16417371]]
0.1
69
[[ 3372.243269    1792.96446041  4090.35333696]
 [ 2210.22362835 51880.04756648  1710.4669927 ]
 [  853.71167881  1098.24404783 79743.38289433]
 [21161.70920799   868.13540342  2389.92767845]
 [13222.26741256  1363.3493022   4141.0611427 ]
 [ 1959.74663182  2279.74931819  1232.62913043]]
0.1
70
[[ 3362.25315452  1785.65489158  4079.28170167]
 [ 2205.80688816 51858.38906319  1706.46101913]
 [  853.43263946

C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:48: RuntimeWarning: invalid value encountered in log
  bottom = - ((beta**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:64: RuntimeWarning: overflow encountered in exp
  prop_mu = np.abs(np.exp(prop_logmu))
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:90: RuntimeWarning: overflow encountered in exp
  acc_prob = np.exp(top - bottom)
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:88: RuntimeWarning: overflow encountered in power
  top = - (prop**2 / (2*tau)) + np.log(prop**y[k, l]) - som_des_logs(y[k][l])
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:89: RuntimeWarning: overflow encountered in scalar power
  bottom = - (mu[k, l]**2/ (2*tau)) + np.log(mu[k, l]**y[k, l]) - som_des_logs(y[k][l])
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_13856\2972978751.py:90: RuntimeWarning: inval

[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1083
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1084
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1085
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1086
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1087
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1088
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1089
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1090
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1091
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
0.1
1092


In [164]:

# Implémentation de la méthode MCMC (Hastings within Gibbs)
def model(y, x, nchain, alpha, beta, gamma, tau, lambda_):
    n = len(x)
    m = y.shape[1]
    res = np.zeros((nchain, 4))
    for k in range(nchain):
        for i in range(n):
            for j in range(m):
                mu_ij = np.exp(alpha + beta * np.log(x[i] + 10) + gamma * x[i] + lambda_[i, j])
                lambda_[i, j] = np.random.normal(0, np.sqrt(1/tau))
                y[i, j] = np.random.poisson(mu_ij)
        alpha = np.random.normal(alpha_star, np.sqrt(1/1e6))
        beta = np.random.normal(0, np.sqrt(1/1e6))
        gamma = np.random.normal(0, np.sqrt(1/1e6))
        tau = np.random.gamma(0.001 + 9 * n * m, 0.001 + 0.5 * np.sum(lambda_**2))
        res[k] = [alpha, beta, gamma, 1/np.sqrt(tau)]
    return res

results = model(y, x, 10000, alpha_star, beta, gamma, tau, lambda_)

# Récupération des résultats
burn_in = 1000
results_post_burn_in = results[burn_in:, :]
moychain = np.mean(results_post_burn_in, axis=0) # moyenne
sdchain = np.std(results_post_burn_in, axis=0) # écart type

print("Alpha est estimé à", moychain[0], "avec un écart-type de", sdchain[0])
print("Beta est estimé à", moychain[1], "avec un écart-type de", sdchain[1])
print("Gamma est estimé à", moychain[2], "avec un écart-type de", sdchain[2])
print("Sigma est estimé à", moychain[3], "avec un écart-type de", sdchain[3])

# Visualisation des résultats
plt.figure(figsize=(10, 10))

# Allure des densités pour Alpha, Beta, Gamma et Sigma
plt.subplot(2, 2, 1)
plt.title("Density de Alpha")
plt.hist(results_post_burn_in[:, 0], bins=30, density=True)
plt.subplot(2, 2, 2)
plt.title("Density de Beta")
plt.hist(results_post_burn_in[:, 1], bins=30, density=True)
plt.subplot(2, 2, 3)
plt.title("Density de Gamma")
plt.hist(results_post_burn_in[:, 2], bins=30, density=True)
plt.subplot(2, 2, 4)
plt.title("Density de Sigma")
plt.hist(results_post_burn_in[:, 3], bins=30, density=True)

plt.tight_layout()
plt.show()


NameError: name 'lambda_' is not defined

In [2]:
mean = [moychain[0], moychain[1], moychain[2], moychain[3]]
sd = [sdchain[0], sdchain[1], sdchain[2], sdchain[3]]

In [218]:
#Baseline here indicates Breslow and Clayton estimates
def plot_parameter_comparison(params_df):
    ind = np.arange(len(params_df))  # the x locations for the groups
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(ind - width / 2, params_df['mean'], width, label='Model', yerr=params_df['std'])
    rects2 = ax.bar(ind + width / 2, params_df['mean_base'], width, label='Baseline', yerr=params_df['std_base'])

    # Add some text for labels, title, and custom x-axis tick labels
    ax.set_ylabel('Values')
    ax.set_title('Model vs. baseline Parameter Comparison')
    ax.set_xticks(ind)
    ax.set_xticklabels(['alpha', 'beta', 'gamma', 'sigma'])
    ax.legend()

    plt.show()

#results_summary = results.describe().T
import pandas as pd
params_df = pd.DataFrame()
params_df['mean'] = mean
params_df['std'] = sd
#Breslow and Clayton estimates
params_df['mean_base'] = [2.193, 0.3059, 69.577*(10**(-4)), 0.2608]
params_df['std_base'] = [0.3874, 0.1054, 4.525*(10**(-4)), 0.08077]
params_df.reset_index(drop=True, inplace=True)

plot_parameter_comparison(params_df)

NameError: name 'mean' is not defined